In [3]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.utils import np_utils
import numpy as np
import pandas as pd
from keras import initializers
from sklearn.model_selection import train_test_split
import requests
import datetime

def init_weights(shape, name=None):
    return initializers.normal(shape, scale=0.01, name=name)

Using TensorFlow backend.


In [35]:

API_ENDPOINT_URL =  'http://lauzhack.sqpub.ch/histo_prices'
r = requests.get(API_ENDPOINT_URL)
raw_data = r.text.split('\n')
data_series = pd.Series(raw_data)
data_series = data_series.str.split(' ')
data = pd.DataFrame(data_series.values.tolist(), index=data_series.index)
data.columns = ['timestamp', 'price']
data['timestamp'] = pd.to_datetime(data.timestamp)
time_stamps = data['timestamp']
data.set_index('timestamp', inplace=True)
data['price'] = data['price'].astype(float)
mean_minute = data.groupby(pd.Grouper(freq='60s')).mean()[2881:].dropna()
mean_minute


input_step_size = 50
output_size = 30
sliding_window = False

prices= data.loc[:,'price'].values
times = data.loc[:,'price'].values
prices.shape


# In[31]:


outputs = []
inputs = []
output_times = []
input_times = []
if sliding_window:
    for i in range(len(prices)-input_step_size-output_size):
        inputs.append(prices[i:i + input_step_size])
        input_times.append(times[i:i + input_step_size])
        outputs.append(prices[i + input_step_size: i + input_step_size+ output_size])
        output_times.append(times[i + input_step_size: i + input_step_size+ output_size])
else:
    for i in range(0,len(prices)-input_step_size-output_size, input_step_size):
        inputs.append(prices[i:i + input_step_size])
        input_times.append(times[i:i + input_step_size])
        outputs.append(prices[i + input_step_size: i + input_step_size+ output_size])
        output_times.append(times[i + input_step_size: i + input_step_size+ output_size])
inputs= np.array(inputs)
outputs= np.array(outputs)
output_times = np.array(output_times)
input_times = np.array(input_times)


In [36]:
import numpy as np
import pandas as pd
 
class PastSampler:
    '''
    Forms training samples for predicting future values from past value
    '''
     
    def __init__(self, N, K, sliding_window = True):
        '''
        Predict K future sample using N previous samples
        '''
        self.K = K
        self.N = N
        self.sliding_window = sliding_window
 
    def transform(self, A):
        M = self.N + self.K     #Number of samples per row (sample + target)
        #indexes
        if self.sliding_window:
            I = np.arange(M) + np.arange(A.shape[0] - M + 1).reshape(-1, 1)
        else:
            if A.shape[0]%M == 0:
                I = np.arange(M)+np.arange(0,A.shape[0],M).reshape(-1,1)
                
            else:
                I = np.arange(M)+np.arange(0,A.shape[0] -M,M).reshape(-1,1)
            
        B = A[I].reshape(-1, M * A.shape[1], A.shape[2])
        ci = self.N * A.shape[1]    #Number of features per sample
        return B[:, :ci], B[:, ci:] #Sample matrix, Target matrix



#Columns of price data to use
columns = ['price']
data = data.loc[:,columns]
original_data = data.loc[:,columns]

In [50]:

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# normalization
for c in columns:
    data[c] = scaler.fit_transform(data[c].values.reshape(-1,1))
    
    
    
#Features are input sample dimensions(channels)
A = np.array(data)[:,None,:]
original_A = np.array(original_data)[:,None,:]
time_stamps = np.array(time_stamps)[:,None,None]

#Make samples of temporal sequences of pricing data (channel)
NPS, NFS = 256, 16         #Number of past and future samples
ps = PastSampler(NPS, NFS, sliding_window=False)
B, Y = ps.transform(A)
input_times, output_times = ps.transform(time_stamps)
original_B, original_Y = ps.transform(original_A)

file_name='bitcoin2015to2017_close.h5'

import h5py
with h5py.File(file_name, 'w') as f:
    f.create_dataset("inputs", data = B)
    f.create_dataset('outputs', data = Y)
    f.create_dataset("input_times", data = np.string_(input_times))
    f.create_dataset('output_times', data = np.string_(output_times))
    f.create_dataset("original_datas", data=np.array(original_data))
    f.create_dataset('original_inputs',data=original_B)
    f.create_dataset('original_outputs',data=original_Y)



In [54]:

import pandas as pd
import numpy as numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D, LeakyReLU, PReLU
from keras.utils import np_utils
from keras.callbacks import CSVLogger, ModelCheckpoint
import h5py
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session




with h5py.File(''.join(['bitcoin2015to2017_close.h5']), 'r') as hf:
    datas = hf['inputs'].value
    labels = hf['outputs'].value


output_file_name='bitcoin2015to2017_close_CNN_2_relu'

step_size = datas.shape[1]
batch_size= 8
nb_features = datas.shape[2]

epochs = 100

#split training validation
training_size = int(0.8* datas.shape[0])
training_datas = datas[:training_size,:]
training_labels = labels[:training_size,:]
validation_datas = datas[training_size:,:]
validation_labels = labels[training_size:,:]
#build model

# 2 layers
model = Sequential()


model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=3, filters=8, kernel_size=20))
model.add(Dropout(0.5))
model.add(Conv1D( strides=4, filters=nb_features, kernel_size=16))

'''
# 3 Layers
model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=3, filters=8, kernel_size=8))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=8))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D( strides=2, filters=nb_features, kernel_size=8))
# 4 layers
model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=2, filters=8, kernel_size=2))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=2))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=2))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D( strides=2, filters=nb_features, kernel_size=2))
'''

model.compile(loss='mse', optimizer='adam')
model.fit(training_datas, training_labels,verbose=1, batch_size=batch_size,validation_data=(validation_datas,validation_labels), epochs = epochs)


Train on 22 samples, validate on 6 samples
Epoch 1/100
22/22 [==============================] - 0s 14ms/step - loss: 0.1479 - val_loss: 0.0035
Epoch 2/100
22/22 [==============================] - 0s 329us/step - loss: 0.1061 - val_loss: 0.0021
Epoch 3/100
22/22 [==============================] - 0s 369us/step - loss: 0.0692 - val_loss: 0.0015
Epoch 4/100
22/22 [==============================] - 0s 491us/step - loss: 0.0749 - val_loss: 0.0014
Epoch 5/100
22/22 [==============================] - 0s 509us/step - loss: 0.0461 - val_loss: 0.0015
Epoch 6/100
22/22 [==============================] - 0s 858us/step - loss: 0.0423 - val_loss: 0.0017
Epoch 7/100
22/22 [==============================] - 0s 550us/step - loss: 0.0431 - val_loss: 0.0017
Epoch 8/100
22/22 [==============================] - 0s 502us/step - loss: 0.0387 - val_loss: 0.0016
Epoch 9/100
22/22 [==============================] - 0s 522us/step - loss: 0.0346 - val_loss: 0.0015
Epoch 10/100
22/22 [=============================

22/22 [==============================] - 0s 624us/step - loss: 0.0079 - val_loss: 0.0015
Epoch 82/100
22/22 [==============================] - 0s 513us/step - loss: 0.0089 - val_loss: 0.0016
Epoch 83/100
22/22 [==============================] - 0s 627us/step - loss: 0.0070 - val_loss: 0.0016
Epoch 84/100
22/22 [==============================] - 0s 481us/step - loss: 0.0074 - val_loss: 0.0018
Epoch 85/100
22/22 [==============================] - 0s 476us/step - loss: 0.0065 - val_loss: 0.0021
Epoch 86/100
22/22 [==============================] - 0s 564us/step - loss: 0.0083 - val_loss: 0.0022
Epoch 87/100
22/22 [==============================] - 0s 480us/step - loss: 0.0090 - val_loss: 0.0020
Epoch 88/100
22/22 [==============================] - 0s 425us/step - loss: 0.0077 - val_loss: 0.0017
Epoch 89/100
22/22 [==============================] - 0s 399us/step - loss: 0.0071 - val_loss: 0.0018
Epoch 90/100
22/22 [==============================] - 0s 523us/step - loss: 0.0062 - val_loss: 

In [69]:
predicted = model.predict(validation_datas)
predicted


array([[[0.2333922 ],
        [0.23536345],
        [0.23784117],
        [0.2378177 ],
        [0.24119873],
        [0.24708173],
        [0.24626613],
        [0.24562036],
        [0.24333675],
        [0.23850518],
        [0.23936404],
        [0.23516054],
        [0.22236314],
        [0.2201446 ],
        [0.2086715 ],
        [0.19840674]],

       [[0.17188847],
        [0.15913966],
        [0.14431202],
        [0.13650128],
        [0.1310913 ],
        [0.13013579],
        [0.13477358],
        [0.12859496],
        [0.13095059],
        [0.12634866],
        [0.13155825],
        [0.1366965 ],
        [0.13634083],
        [0.1419087 ],
        [0.14959326],
        [0.14830473]],

       [[0.11832517],
        [0.11316681],
        [0.12172216],
        [0.12848933],
        [0.13104074],
        [0.14549646],
        [0.13797167],
        [0.14005187],
        [0.13876306],
        [0.14434874],
        [0.14887872],
        [0.15233627],
        [0.15859784],
      

In [72]:

import pandas as pd
import numpy as numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Reshape
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import np_utils
from keras.layers import LSTM, LeakyReLU
from keras.callbacks import CSVLogger, ModelCheckpoint
import h5py
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session




with h5py.File(''.join(['bitcoin2015to2017_close.h5']), 'r') as hf:
    datas = hf['inputs'].value
    labels = hf['outputs'].value




step_size = datas.shape[1]
units= 50
second_units = 30
batch_size = 8
nb_features = datas.shape[2]
epochs = 100
output_size=16
#split training validation
training_size = int(0.8* datas.shape[0])
training_datas = datas[:training_size,:]
training_labels = labels[:training_size,:,0]
validation_datas = datas[training_size:,:]
validation_labels = labels[training_size:,:,0]


#build model
model = Sequential()
model.add(LSTM(units=units,activation='tanh', input_shape=(step_size,nb_features),return_sequences=False))
model.add(Dropout(0.8))
model.add(Dense(output_size))
model.add(LeakyReLU())
model.compile(loss='mse', optimizer='adam')
model.fit(training_datas, training_labels, batch_size=batch_size,validation_data=(validation_datas,validation_labels), epochs = epochs)


Train on 22 samples, validate on 6 samples
Epoch 1/100
22/22 [==============================] - 2s 69ms/step - loss: 0.1935 - val_loss: 0.0157
Epoch 2/100
22/22 [==============================] - 0s 16ms/step - loss: 0.1839 - val_loss: 0.0132
Epoch 3/100
22/22 [==============================] - 0s 16ms/step - loss: 0.1681 - val_loss: 0.0109
Epoch 4/100
22/22 [==============================] - 0s 14ms/step - loss: 0.1573 - val_loss: 0.0086
Epoch 5/100
22/22 [==============================] - 0s 16ms/step - loss: 0.1364 - val_loss: 0.0067
Epoch 6/100
22/22 [==============================] - 0s 14ms/step - loss: 0.1167 - val_loss: 0.0057
Epoch 7/100
22/22 [==============================] - 0s 12ms/step - loss: 0.1271 - val_loss: 0.0057
Epoch 8/100
22/22 [==============================] - 0s 14ms/step - loss: 0.1415 - val_loss: 0.0050
Epoch 9/100
22/22 [==============================] - 0s 15ms/step - loss: 0.1190 - val_loss: 0.0040
Epoch 10/100
22/22 [==============================] - 0s 

22/22 [==============================] - 0s 13ms/step - loss: 0.0319 - val_loss: 0.0015
Epoch 80/100
22/22 [==============================] - 0s 12ms/step - loss: 0.0196 - val_loss: 0.0012
Epoch 81/100
22/22 [==============================] - 0s 13ms/step - loss: 0.0339 - val_loss: 0.0011
Epoch 82/100
22/22 [==============================] - 0s 14ms/step - loss: 0.0201 - val_loss: 0.0010
Epoch 83/100
22/22 [==============================] - 0s 12ms/step - loss: 0.0327 - val_loss: 0.0011
Epoch 84/100
22/22 [==============================] - 0s 14ms/step - loss: 0.0258 - val_loss: 0.0015
Epoch 85/100
22/22 [==============================] - 0s 15ms/step - loss: 0.0227 - val_loss: 0.0020
Epoch 86/100
22/22 [==============================] - 0s 13ms/step - loss: 0.0265 - val_loss: 0.0026
Epoch 87/100
22/22 [==============================] - 0s 14ms/step - loss: 0.0251 - val_loss: 0.0023
Epoch 88/100
22/22 [==============================] - 0s 13ms/step - loss: 0.0254 - val_loss: 0.0019
Epo

In [73]:
predicted = model.predict(validation_datas)
predicted


array([[0.17281975, 0.16930988, 0.18131165, 0.1885862 , 0.15549396,
        0.16543642, 0.17254569, 0.18146437, 0.16966446, 0.16211414,
        0.19566512, 0.1770158 , 0.18560435, 0.18399076, 0.19063601,
        0.22183488],
       [0.13400082, 0.127956  , 0.14055097, 0.15238659, 0.11253364,
        0.12428259, 0.13279374, 0.14282727, 0.13458249, 0.1215101 ,
        0.1618751 , 0.13869512, 0.15199563, 0.14751366, 0.1554972 ,
        0.19622879],
       [0.14686441, 0.14172186, 0.1539762 , 0.16472931, 0.12672871,
        0.13748255, 0.14620835, 0.1558702 , 0.14635432, 0.13517892,
        0.17285267, 0.1511579 , 0.16347384, 0.15991125, 0.16732684,
        0.2046789 ],
       [0.15890853, 0.15470517, 0.16647899, 0.17664322, 0.13997039,
        0.1493381 , 0.1590403 , 0.16832055, 0.15757811, 0.14824653,
        0.18289381, 0.16252826, 0.17463998, 0.17189479, 0.17860083,
        0.21260113],
       [0.14076911, 0.13498537, 0.14771497, 0.15815452, 0.12008433,
        0.13232093, 0.13923952, 